In [16]:
import pandas as pd
from pytz import timezone
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import ast
import re
from dateutil import parser
from datetime import datetime, timedelta
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [17]:
import csv

# Specify the file path of the CSV file
file_path = 'top_sellers.csv'

# Open the CSV file in read mode and read the data
with open(file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    data = next(reader)

# 'data' will contain the list of strings from the CSV file
data = list(map(int, data))

In [18]:
## 2023-06-28 start from here
merged_df = pd.read_csv('merged_df3834.csv')

In [19]:
merged_df

,product_id,timestamp,year,month,day,hour,day_of_week,hourly_sales,instock_fraction
0,0,2022-01-01 00:00:00,2022,1,1,0,5,0.0,1.0
1,0,2022-01-01 01:00:00,2022,1,1,1,5,0.0,1.0
2,0,2022-01-01 02:00:00,2022,1,1,2,5,0.0,1.0
3,0,2022-01-01 03:00:00,2022,1,1,3,5,0.0,1.0
4,0,2022-01-01 04:00:00,2022,1,1,4,5,0.0,1.0
...,...,...,...,...,...,...,...,...,...
38219875,999,2022-12-31 19:00:00,2022,12,31,19,5,0.0,1.0
38219876,999,2022-12-31 20:00:00,2022,12,31,20,5,0.0,1.0
38219877,999,2022-12-31 21:00:00,2022,12,31,21,5,0.0,1.0
38219878,999,2022-12-31 22:00:00,2022,12,31,22,5,0.0,1.0


In [20]:
merged_df['stockout_dum'] = (1 - merged_df['instock_fraction']).gt(1/60).astype(int)

In [21]:
df_cooccurrence_of667 = pd.read_csv("final_co_occurrence_ratio_for_3834.csv")

In [22]:
df_sorted = df_cooccurrence_of667.sort_values('co_occurrence_ratio_wrt_product_1', ascending=False)

top = 11
# Get the 40 biggest 'idpx' values
top_biggest = df_sorted[1:top]['product_id_2']

top_biggest

1      610
2     1610
3     2668
4      752
5      106
6     2571
7      463
8      523
9     4144
10     252
Name: product_id_2, dtype: int64

In [23]:
merged_df['com_dum']=0
merged_df.loc[merged_df['product_id'].isin(top_biggest),'com_dum']=1

merged_df['stockout_com_interaction'] = merged_df['stockout_dum']*merged_df['com_dum']
merged_df

,product_id,timestamp,year,month,day,hour,day_of_week,hourly_sales,instock_fraction,stockout_dum,com_dum,stockout_com_interaction
0,0,2022-01-01 00:00:00,2022,1,1,0,5,0.0,1.0,0,0,0
1,0,2022-01-01 01:00:00,2022,1,1,1,5,0.0,1.0,0,0,0
2,0,2022-01-01 02:00:00,2022,1,1,2,5,0.0,1.0,0,0,0
3,0,2022-01-01 03:00:00,2022,1,1,3,5,0.0,1.0,0,0,0
4,0,2022-01-01 04:00:00,2022,1,1,4,5,0.0,1.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
38219875,999,2022-12-31 19:00:00,2022,12,31,19,5,0.0,1.0,0,0,0
38219876,999,2022-12-31 20:00:00,2022,12,31,20,5,0.0,1.0,0,0,0
38219877,999,2022-12-31 21:00:00,2022,12,31,21,5,0.0,1.0,0,0,0
38219878,999,2022-12-31 22:00:00,2022,12,31,22,5,0.0,1.0,0,0,0


In [24]:
merged_df = merged_df[merged_df['product_id'].isin(data)]

merged_df.shape

(3504000, 12)

# Regression

$ln(hourly\_sales_{p,t}) = \beta_0 + \beta_1 * stockout\_dum\_t + \beta_2 * comp\_dum\_p+\beta_3 * stockout\_dum\_t * comp\_dum\_p + \delta\_prod(p) +\gamma\_m(t) + \theta\_d(t) + \alpha\_dow(t) + \delta\_h(t) + ε_{p,t}$

Y_{p,t}:  represents the outcome variable which is log hourly sales for each product p at time t.

$stockout\_dum\_t$: is a dummy variable that equals 1 if the order was placed when stockout.

$comp\_dum\_p$: is a dummy variable that equals 1 if product p is complementary of the independant variable.

$\delta\_prod(p)$: fixed effect of products.

$\gamma\_m(t)$: fixed effect of month.

$\theta\_d(t)$: fixed effect of day.

$\alpha\_dow(t)$: fixed effect of day of week.

$\delta\_h(t)$: fixed effect of hour.

ε_it represents the error term.

In [25]:
df_regression = merged_df.copy()

df_regression = df_regression[df_regression['product_id']!=3834]

In [26]:
df_regression_2022 = df_regression[(df_regression['year']==2022)]

In [27]:
df_regression_2022 = df_regression_2022[df_regression_2022['month'].isin([1,2,3,4,5,6])]
#df_regression_2022 = df_regression_2022[df_regression_2022['month'].isin([2])]

In [28]:
df_regression_2022 = df_regression_2022.drop(['timestamp','instock_fraction','year'], axis=1)

In [29]:
month_dummies = pd.get_dummies(df_regression_2022['month'], prefix='month', drop_first=True)
#day_dummies = pd.get_dummies(df_regression_2022['day'], prefix='day', drop_first=True)
day_of_week_dummies = pd.get_dummies(df_regression_2022['day_of_week'], prefix='day', drop_first=True)
hour_dummies = pd.get_dummies(df_regression_2022['hour'], prefix='hour', drop_first=True)
product_id_dummies = pd.get_dummies(df_regression_2022['product_id'], prefix='product_id', drop_first=True)

# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df_regression_2022, month_dummies, day_of_week_dummies, hour_dummies, product_id_dummies], axis=1)

# Log-transform the dependent variable
df_with_dummies['log_hourly_sales'] = np.log(1 + df_with_dummies['hourly_sales'])

# Create the design matrix by specifying the formula with interaction terms
formula = 'log_hourly_sales ~ stockout_dum + com_dum + stockout_com_interaction + C(product_id) + C(month) + C(day_of_week) + C(hour)'

In [30]:
# Fit the linear regression model
results = smf.ols(formula=formula, data=df_with_dummies).fit()

# Get the summary of the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       log_hourly_sales   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     795.1
Date:                Tue, 04 Jul 2023   Prob (F-statistic):               0.00
Time:                        09:06:58   Log-Likelihood:            -4.4192e+05
No. Observations:             1733256   AIC:                         8.847e+05
Df Residuals:                 1732820   BIC:                         8.901e+05
Df Model:                         435                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [16]:
# Concatenate the dummy variables with the original DataFrame
df_without_dummies = df_regression_2022.copy()

# Log-transform the dependent variable
df_without_dummies['log_hourly_sales'] = np.log(1 + df_without_dummies['hourly_sales'])

df_without_dummies = df_without_dummies.drop(['product_id','month','day','hour','day_of_week','hourly_sales'], axis=1)

# Create the design matrix by specifying the formula with interaction terms
formula_without_dummies = 'log_hourly_sales ~ stockout_dum + com_dum + stockout_com_interaction'

# Fit the linear regression model
results_without_dummies = smf.ols(formula=formula_without_dummies, data=df_without_dummies).fit()
#results_without_dummies = smf.wls(formula = formula_without_dummies, data=df_without_dummies, weights=1/np.square(residuals)).fit(cov_type='HC3')


# Get the summary of the regression results
print(results_without_dummies.summary())

                            OLS Regression Results                            
Dep. Variable:       log_hourly_sales   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1178.
Date:                Mon, 03 Jul 2023   Prob (F-statistic):               0.00
Time:                        12:39:10   Log-Likelihood:            -5.9787e+05
No. Observations:             1733256   AIC:                         1.196e+06
Df Residuals:                 1733252   BIC:                         1.196e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [17]:
from tabulate import tabulate

# Function to determine significance level
def get_significance(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

# Create the summary table in a list format
summary_table_list = []

# Add the coefficient, standard error, and significance level rows
summary_table_list.append(['stockout_dum', '{:.4f} ({:.4f}){}'.format(results.params['stockout_dum'], results.bse['stockout_dum'], get_significance(results.pvalues['stockout_dum'])), '{:.4f} ({:.4f}){}'.format(results_without_dummies.params['stockout_dum'], results_without_dummies.bse['stockout_dum'], get_significance(results_without_dummies.pvalues['stockout_dum']))])
summary_table_list.append(['com_dum', '{:.4f} ({:.4f}){}'.format(results.params['com_dum'], results.bse['com_dum'], get_significance(results.pvalues['com_dum'])), '{:.4f} ({:.4f}){}'.format(results_without_dummies.params['com_dum'], results_without_dummies.bse['com_dum'], get_significance(results_without_dummies.pvalues['com_dum']))])
summary_table_list.append(['stockout_com_interaction', '{:.4f} ({:.4f}){}'.format(results.params['stockout_com_interaction'], results.bse['stockout_com_interaction'], get_significance(results.pvalues['stockout_com_interaction'])), '{:.4f} ({:.4f}){}'.format(results_without_dummies.params['stockout_com_interaction'], results_without_dummies.bse['stockout_com_interaction'], get_significance(results_without_dummies.pvalues['stockout_com_interaction']))])

# Add the inclusion information
summary_table_list.append(['month', 'yes', 'no'])
summary_table_list.append(['day', 'yes', 'no'])
summary_table_list.append(['day_of_week', 'yes', 'no'])
summary_table_list.append(['hour', 'yes', 'no'])

summary_table_list.append(['R-squared', '{:.4f}'.format(results.rsquared), '{:.4f}'.format(results_without_dummies.rsquared)])

summary_table_list.append(['No. Observations', '{:,}'.format(results.nobs), '{:,}'.format(results_without_dummies.nobs)])

# Create the table headers
headers = ['Variable', 'Model_with_FixedEffects', 'Model_without_FixedEffects']

# Print the summary table using tabulate
print(tabulate(summary_table_list, headers, tablefmt="grid"))

In [ ]:
filename = 'summary_table_3834.csv'

# Save the list as a CSV file
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(summary_table_list)

In [10]:
df_regression = merged_df.copy()
df_regression = df_regression[df_regression['product_id']!=3834]
df_regression_2022 = df_regression[(df_regression['year']==2022)]
df_regression_2022 = df_regression_2022[df_regression_2022['month'].isin([1,2,3,4,5,6])]
df_regression_2022 = df_regression_2022.drop(['timestamp','instock_fraction','year'], axis=1)

In [11]:
month_dummies = pd.get_dummies(df_regression_2022['month'], prefix='month', drop_first=True)
day_of_week_dummies = pd.get_dummies(df_regression_2022['day_of_week'], prefix='day', drop_first=True)
hour_dummies = pd.get_dummies(df_regression_2022['hour'], prefix='hour', drop_first=True)
product_id_dummies = pd.get_dummies(df_regression_2022['product_id'], prefix='product_id', drop_first=True)

# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df_regression_2022, month_dummies, day_of_week_dummies, hour_dummies, product_id_dummies], axis=1)

# Log-transform the dependent variable
df_with_dummies['log_hourly_sales'] = np.log(1 + df_with_dummies['hourly_sales'])

# Create the design matrix by specifying the formula with interaction terms
formula = 'log_hourly_sales ~ stockout_dum + com_dum + stockout_com_interaction + C(product_id) + C(month) + C(day_of_week) + C(hour)'

# Add product-by-month fixed effects to the formula
product_month_interaction_terms = ' + C(product_id):C(month)'
formula_with_product_month_effects = formula + product_month_interaction_terms

In [12]:
# Fit the regression model with product-by-month fixed effects
model_with_product_month_effects = smf.ols(formula_with_product_month_effects, data=df_with_dummies).fit()

In [13]:
# Get the summary of the regression results
print(model_with_product_month_effects.summary())

                            OLS Regression Results                            
Dep. Variable:       log_hourly_sales   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     153.0
Date:                Mon, 03 Jul 2023   Prob (F-statistic):               0.00
Time:                        13:09:13   Log-Likelihood:            -2.3610e+05
No. Observations:              861840   AIC:                         4.747e+05
Df Residuals:                  860612   BIC:                         4.890e+05
Df Model:                        1227                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [73]:
summary_table = pd.read_csv('summary_table_3834.csv')

In [74]:
# Get the current headers of the DataFrame
current_headers = list(summary_table.columns)

# Create a new DataFrame with the duplicated headers
header_row = pd.DataFrame([current_headers], columns=summary_table.columns)

# Concatenate the header row with the original summary_table DataFrame
summary_table = pd.concat([header_row, summary_table], ignore_index=True)



In [75]:
headers = ['Variable', 'Model_with_FixedEffects', 'Model_without_FixedEffects']
# Add the headers to the DataFrame
summary_table.columns = headers

In [76]:
# Replace "day" with "product"
summary_table['Variable'] = summary_table['Variable'].replace('day', 'product')

# Rearrange the columns
summary_table = summary_table[['Variable', 'Model_without_FixedEffects', 'Model_with_FixedEffects']]


In [77]:
summary_table

,Variable,Model_without_FixedEffects,Model_with_FixedEffects
0,stockout_dum,0.0073 (0.0012)***,-0.0015 (0.0011)
1,com_dum,0.1184 (0.0020)***,-117042603.4502 (443410914.2873)
2,stockout_com_interaction,-0.0326 (0.0088)***,-0.0326 (0.0080)***
3,month,no,yes
4,product,no,yes
5,day_of_week,no,yes
6,hour,no,yes
7,R-squared,0.0020,0.1664
8,No. Observations,"1,733,256.0","1,733,256.0"


In [78]:
from tabulate import tabulate

# Function to determine significance level
def get_significance(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

# Create the summary table in a list format
summary_table_aux = []

# Add the coefficient, standard error, and significance level rows
summary_table_aux.append(['stockout_dum', '{:.4f} ({:.4f}){}'.format(model_with_product_month_effects.params['stockout_dum'], model_with_product_month_effects.bse['stockout_dum'], get_significance(model_with_product_month_effects.pvalues['stockout_dum']))])
summary_table_aux.append(['com_dum', '{:.4f} ({:.4f}){}'.format(model_with_product_month_effects.params['com_dum'], model_with_product_month_effects.bse['com_dum'], get_significance(model_with_product_month_effects.pvalues['com_dum']))])
summary_table_aux.append(['stockout_com_interaction', '{:.4f} ({:.4f}){}'.format(model_with_product_month_effects.params['stockout_com_interaction'], model_with_product_month_effects.bse['stockout_com_interaction'], get_significance(model_with_product_month_effects.pvalues['stockout_com_interaction']))])

# Add the inclusion information
summary_table_aux.append(['month', 'yes'])
summary_table_aux.append(['product', 'yes'])
summary_table_aux.append(['day_of_week', 'yes'])
summary_table_aux.append(['hour', 'yes'])

summary_table_aux.append(['R-squared', '{:.4f}'.format(model_with_product_month_effects.rsquared)])

summary_table_aux.append(['No. Observations', '{:,}'.format(model_with_product_month_effects.nobs)])

# Create the table headers
headers = ['Variable', 'Model_with_All_FixedEffects']

# Print the summary table using tabulate
print(tabulate(summary_table_aux, headers, tablefmt="grid"))


+--------------------------+---------------------------------+
| Variable                 | Model_with_All_FixedEffects     |
+==========================+=================================+
| stockout_dum             | -0.0022 (0.0012)                |
+--------------------------+---------------------------------+
| com_dum                  | 141997725.9534 (427146594.8403) |
+--------------------------+---------------------------------+
| stockout_com_interaction | -0.0328 (0.0089)***             |
+--------------------------+---------------------------------+
| month                    | yes                             |
+--------------------------+---------------------------------+
| product                  | yes                             |
+--------------------------+---------------------------------+
| day_of_week              | yes                             |
+--------------------------+---------------------------------+
| hour                     | yes                       

In [79]:
summary_table_aux

[['stockout_dum', '-0.0022 (0.0012)'],
 ['com_dum', '141997725.9534 (427146594.8403)'],
 ['stockout_com_interaction', '-0.0328 (0.0089)***'],
 ['month', 'yes'],
 ['product', 'yes'],
 ['day_of_week', 'yes'],
 ['hour', 'yes'],
 ['R-squared', '0.1791'],
 ['No. Observations', '861,840.0']]

In [80]:
summary_table

,Variable,Model_without_FixedEffects,Model_with_FixedEffects
0,stockout_dum,0.0073 (0.0012)***,-0.0015 (0.0011)
1,com_dum,0.1184 (0.0020)***,-117042603.4502 (443410914.2873)
2,stockout_com_interaction,-0.0326 (0.0088)***,-0.0326 (0.0080)***
3,month,no,yes
4,product,no,yes
5,day_of_week,no,yes
6,hour,no,yes
7,R-squared,0.0020,0.1664
8,No. Observations,"1,733,256.0","1,733,256.0"


In [81]:
# Create a new column with the values from summary_table_aux
summary_table['Model_with_All_FixedEffects'] = [row[1] for row in summary_table_aux]

# Print the updated summary_table DataFrame
print(summary_table)


                   Variable Model_without_FixedEffects  \
0              stockout_dum         0.0073 (0.0012)***   
1                   com_dum         0.1184 (0.0020)***   
2  stockout_com_interaction        -0.0326 (0.0088)***   
3                     month                         no   
4                   product                         no   
5               day_of_week                         no   
6                      hour                         no   
7                 R-squared                     0.0020   
8          No. Observations                1,733,256.0   

            Model_with_FixedEffects      Model_with_All_FixedEffects  
0                  -0.0015 (0.0011)                 -0.0022 (0.0012)  
1  -117042603.4502 (443410914.2873)  141997725.9534 (427146594.8403)  
2               -0.0326 (0.0080)***              -0.0328 (0.0089)***  
3                               yes                              yes  
4                               yes                             

In [82]:
# Create a new row for product:month
new_row_product_month = ['product:month', 'no', 'no', 'yes']

# Find the index position of the "hour" row
hour_index = summary_table.index[summary_table['Variable'] == 'hour'][0]

# Insert the new row below the hour row
summary_table = summary_table.append(pd.Series(new_row_product_month, index=summary_table.columns), ignore_index=True)
summary_table = pd.concat([summary_table.iloc[:hour_index + 1], summary_table.iloc[-1:], summary_table.iloc[hour_index + 1:-1]], ignore_index=True)

# Print the updated summary_table DataFrame
print(summary_table)


                   Variable Model_without_FixedEffects  \
0              stockout_dum         0.0073 (0.0012)***   
1                   com_dum         0.1184 (0.0020)***   
2  stockout_com_interaction        -0.0326 (0.0088)***   
3                     month                         no   
4                   product                         no   
5               day_of_week                         no   
6                      hour                         no   
7             product:month                         no   
8                 R-squared                     0.0020   
9          No. Observations                1,733,256.0   

            Model_with_FixedEffects      Model_with_All_FixedEffects  
0                  -0.0015 (0.0011)                 -0.0022 (0.0012)  
1  -117042603.4502 (443410914.2873)  141997725.9534 (427146594.8403)  
2               -0.0326 (0.0080)***              -0.0328 (0.0089)***  
3                               yes                              yes  
4     

<ipython-input-82-642f19906446>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table = summary_table.append(pd.Series(new_row_product_month, index=summary_table.columns), ignore_index=True)


In [83]:
summary_table

,Variable,Model_without_FixedEffects,Model_with_FixedEffects,Model_with_All_FixedEffects
0,stockout_dum,0.0073 (0.0012)***,-0.0015 (0.0011),-0.0022 (0.0012)
1,com_dum,0.1184 (0.0020)***,-117042603.4502 (443410914.2873),141997725.9534 (427146594.8403)
2,stockout_com_interaction,-0.0326 (0.0088)***,-0.0326 (0.0080)***,-0.0328 (0.0089)***
3,month,no,yes,yes
4,product,no,yes,yes
5,day_of_week,no,yes,yes
6,hour,no,yes,yes
7,product:month,no,no,yes
8,R-squared,0.0020,0.1664,0.1791
9,No. Observations,"1,733,256.0","1,733,256.0","861,840.0"
